In [49]:
import numpy as np
import nltk
import spacy
from spacy_langdetect import LanguageDetector

In [41]:
import time, sys
from IPython.display import clear_output

def update_progress(progress):
    bar_length = 20
    if isinstance(progress, int):
        progress = float(progress)
    if not isinstance(progress, float):
        progress = 0
    if progress < 0:
        progress = 0
    if progress >= 1:
        progress = 1

    block = int(round(bar_length * progress))

    clear_output(wait = True)
    text = "Progress: [{0}] {1:.1f}%".format( "#" * block + "-" * (bar_length - block), progress * 100)
    print(text)


In [3]:
!ls

 c_g_rev_cleaner.py     geckodriver.log         progress.txt
'Cleaning Pipe.ipynb'   g_rev_scrape.py         samples.py
 data		       'NLP experiment.ipynb'   samples.txt


In [5]:
path = 'data/cleaned.npy'
arr = np.load(path, allow_pickle = True)

In [7]:
arr.shape

(40627, 4)

In [20]:
words = arr[:,3]

In [35]:
b = TextBlob(words[0])

In [37]:
b.detect_language()

'de'

In [63]:
nlp = spacy.load("en")
nlp.add_pipe(LanguageDetector(), name="language_detector", last=True)

lang = []
for idx, w in enumerate(words):
    if len(w)<4:
        lang.append(-1)
    else:
        doc = nlp(w)
        lang.append(doc._.language['language'])
    progress_bar = idx
    update_progress((idx+1) / len(words))
    




Progress: [####################] 100.0%


In [65]:
len(words)

40627

In [72]:
l =np.array(lang)
l.shape
l =l.reshape(-1,1)

In [77]:
arr2 = np.concatenate((arr,l), axis = 1 )

In [78]:
arr2.shape

(40627, 5)

In [81]:
arr2[3]

array([5, '1927303-mary', ('Paperback 9781435756342',),
       'Live love laugh a Lot is a special book.Interesting characters make you laugh- you must take a look.Voicing their opinions in humorous forms moving line by line.Entertaining to such an extent- you must take your time.Love the way Jacquelyn speaks through Sly Fox.Over and over you have to laugh as he comes out of his box.Very sly he tries to be from behind his cloth.Excitement on each page you must laugh a lot.Laughter rings out as Ingrid does her stuff.A woman who could finally say, “Enough is Enough”.Use me Lord Jacquelyn says on page fifty-eight.God’s blessings comes in many ways Jacquelyn points out to us.Honesty and humor you will find within these pages so much.A great read and must for every home.Live Love Laugh a lotOh how humorousThis book is written by Jacquelyn Sturge ',
       'en'], dtype=object)

In [82]:
np.save("data/clean_and_lang", arr2) ## might need to do a mongo thing
